In [14]:
from training_utils import create_siamese_dataset
from ffcv.writer import DatasetWriter
import numpy as np
import torch
from torch.utils.data import TensorDataset

In [15]:
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import ToTensor, ToDevice
from ffcv.fields.decoders import IntDecoder, NDArrayDecoder, FloatDecoder
from ffcv.writer import DatasetWriter
from ffcv.fields import IntField, FloatField, TorchTensorField

In [17]:
mouse_records  = np.load('data/mouse_record_interpolated.npy')


In [68]:
mr0 = torch.from_numpy(mouse_records).permute(0,2,1)[:1000]
mr0 = mr0.double()


In [70]:
mr0.shape

torch.Size([1000, 2, 30])

In [46]:
mr0.reshape((len(mr0), 60)).shape

torch.Size([1000, 60])

In [71]:
td = TensorDataset(mr0, torch.ones(len(mr0)))

In [72]:
td[0]

(tensor([[0.8950, 0.7687, 0.6545, 0.6347, 0.6384, 0.6459, 0.6500, 0.6500, 0.6500,
          0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500,
          0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500, 0.6500,
          0.6500, 0.6500, 0.6500],
         [0.4933, 0.8799, 0.9792, 0.6603, 0.6312, 0.6135, 0.6133, 0.6133, 0.6133,
          0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133,
          0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133, 0.6133,
          0.6133, 0.6133, 0.6133]], dtype=torch.float64),
 tensor(1.))

In [3]:
path = "data/resnet50_losses_0.npy"
tensor_dataset = create_siamese_dataset(path, 1.)

In [10]:
tensor_dataset[0][0].shape

torch.Size([2, 30])

In [73]:
write_path = "data/siamese_dataset.beton"
writer = DatasetWriter(write_path, {
    # Tune options to optimize dataset size, throughput at train-time
    'mr0': TorchTensorField(
        dtype=torch.float64,
        shape=(30, 2),
    ),
    # 'mr1': TorchTensorField(
    #     dtype=torch.float64,
    #     shape=(60, 1),
    # ),
    'target': IntField(),
    # 't0': FloatField(),
    # 't1': FloatField(),
    # 'w0': FloatField(),
    # 'w1': FloatField(),
})
writer.from_indexed_dataset(td)
print(f"Dataset written to { write_path}")









100%|██████████| 1000/1000 [00:00<00:00, 9963.50it/s]


Dataset written to data/siamese_dataset.beton


In [8]:
# Your dataset (`torch.utils.data.Dataset`) of (image, label) pairs
my_dataset = np.ones((100, 128))
dataset = TensorDataset(torch.from_numpy(my_dataset))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10)
for i, (b) in enumerate(dataloader):
    print(i, b[0].shape)
    break

In [40]:
from ffcv.writer import DatasetWriter
from ffcv.fields import RGBImageField, IntField, FloatField, TorchTensorField

# Your dataset (`torch.utils.data.Dataset`) of (image, label) pairs
mr0 = np.ones((100, 60))
mr1 = np.ones((100, 60))
t = np.ones((100, 1), dtype=int)

dataset_better = TensorDataset(torch.from_numpy(mr0), torch.from_numpy(mr1), torch.from_numpy(t))
dataloader_better = torch.utils.data.DataLoader(dataset_better, batch_size=10)
for i, (m0, m1, t_) in enumerate(dataloader_better):
    print(i, m0.shape, m1.shape, t_.shape)
    break

0 torch.Size([10, 60]) torch.Size([10, 60]) torch.Size([10, 1])


In [41]:
len(dataset_better), dataset_better[0]

(100,
 (tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1.], dtype=torch.float64),
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1.], dtype=torch.float64),
  tensor([1])))

In [42]:
write_path = "test.beton"
# Pass a type for each data field
writer = DatasetWriter(write_path, {
    # Tune options to optimize dataset size, throughput at train-time
    'mr0': TorchTensorField(
        dtype=torch.float64,
        shape=(60, 1),
    ),
    'mr1': TorchTensorField(
        dtype=torch.float64,
        shape=(60, 1),
    ),
    'target': IntField()
})

# Write dataset
writer.from_indexed_dataset(dataset_better)






100%|██████████| 100/100 [00:00<00:00, 996.46it/s]


In [47]:
from ffcv.loader import Loader, OrderOption
from ffcv.transforms import ToTensor, ToDevice
from ffcv.fields.decoders import IntDecoder, NDArrayDecoder

# Data decoding and augmentation
mr_pipeline = [ NDArrayDecoder(), ToTensor()]
target_pipeline = [IntDecoder(), ToTensor()]

# Pipeline for each data field
pipelines = {
    'mr0': mr_pipeline,
    'mr1': mr_pipeline,
    'target': target_pipeline
}

# Replaces PyTorch data loader (`torch.utils.data.Dataloader`)
loader = Loader(write_path, batch_size=8, num_workers=1,
                order=OrderOption.RANDOM, pipelines=pipelines)


In [52]:
for i, (mr0, mr1, t) in enumerate(loader):
    print(mr0.shape, mr1.shape, t.shape)
    break

torch.Size([8, 60, 1]) torch.Size([8, 60, 1]) torch.Size([8, 1])


In [ ]:

write_path = 'test.beton'

# Pass a type for each data field
writer = DatasetWriter(write_path, {
    # Tune options to optimize dataset size, throughput at train-time
    'image': RGBImageField(
        max_resolution=256
    ),
    'label': IntField()
})

# Write dataset
writer.from_indexed_dataset(my_dataset)

In [ ]:
sh_path = "/scratch_local/owl156-462029/NeglectedFreeLunch/data/resnet50_losses_0.npy",
margin=1
arr = create_siamese_data(sh_path, margin)
np.save(args.destiny_path, arr)